In [1]:
from conllu import parse
from nltk.corpus import conll2000
from collections import Counter, defaultdict

# Вспомогательные функции с семинара

In [2]:
# Функция для расчёта точности теггера
def accuracy(test_sents, postagger):
    errors = 0
    length = 0
    for sent in test_sents:
        length += len(sent)
        sent, real_tags = zip(*sent)
        my_tags = postagger.tag(sent)
        for i in range(len(my_tags)):
            if my_tags[i][1] != real_tags[i]:
                errors += 1
    return 1 - errors / length

# Нормализатор для получения распределения вероятностей из частот
class BaseNormalizer:
    def normalize(self, counter):
        sum_ = sum(counter.values())
        for token in counter:
            counter[token] /= sum_

# Реализация Bigram POS tagger

In [3]:
# Эмиссионная модель остается как в семинаре,
# тк логика присвоения вероятности тега тому или иному слову
# не зависит от типа теггера

class EmissionModel:
    def __init__(self, tagged_sents, normalizer=BaseNormalizer()):
        self.normalizer = normalizer
        self.model = defaultdict(Counter)
        # self.model будет иметь вид 
        # defaultdict({
        #     'tag_1': Counter({'word_1': 0.3, 'word_2': 0.7}),
        #     'tag_2': Counter({'word_1': 0.6, 'word_3': 0.3 ...})
        # })
        for sent in tagged_sents:
            for word, tag in sent:
                self.model[tag][word] += 1
        self.add_unk_token()
        for tag in self.model:
            self.normalizer.normalize(self.model[tag])
        
    def add_unk_token(self):
        # Для каждого тега добавим одинаковую вероятность быть приписанным любому слову, которого нет в модели
        for tag in self.model:
            self.model[tag]['UNK'] = 0.1
        
    def tags(self):
        # Добавим возможность возвращать все теги, которые есть в модели
        return self.model.keys()
    
    def __getitem__(self, tag):
        # Все слова для данного тега
        return self.model[tag]
    
    def __call__(self, word, tag):
        # Самое интересное - вероятность P(word|tag)
        if word not in self[tag]:
            return self[tag]['UNK']
        return self[tag][word]

In [4]:
# Отвечает за определение вероятности P(tag_1|tag_2)

class TransitionModel:
    def __init__(self, tag_seqs, normalizer=BaseNormalizer()):
        self.normalizer = normalizer
        # Хранит вероятности P(tag_1|tag_2) в виде:
        # defaultdict(
        # 'tag_1': Counter({
        #     'tag_1': 0.34, 'tag_2': 0.1, ...
        # }),
        # 'tag_2': Counter({
        #     'tag_1': 0.34, 'tag_2': 0.1, ...
        # }), ...)
        self.model = defaultdict(Counter)

        for sent in tag_seqs:
            for i, tag in enumerate(sent):
                if i == 0:
                    # Счетчик для тега, встречающегося в начале предложения
                    self.model['START'][sent[0]] += 1
                    continue
                # Счетчик для остальных тегов в предложении
                self.model[sent[i - 1]][tag] += 1

        # Для каждого тега получаем значение распределения вероятностей из частот появления тегов:
        for tag in self.model:
            self.normalizer.normalize(self.model[tag])

    def tags(self):
        return self.model.keys()

    def __getitem__(self, tag):
        # Все теги перед текущим тегом
        return self.model[tag]
    
    def __call__(self, tag, prev_tag=None):
        # Возвращает вероятность P(tag_1|tag_2)
        if not prev_tag:
            return self.model['START'][tag]
        return self.model[prev_tag][tag]

In [5]:
# Сопоставляет последовательность слов с последовательностью тегов.

class BigramPOSTagger:
    def __init__(self, emission_model, transition_model):
        self.em = emission_model
        self.tm = transition_model

    def tag(self, sent):
        # Для списка слов возвращаем список пар (слово, тег)
        # Для каждого слова проходимся по всем тегам
        # И выбираем максимум по формуле (8.18) из учебника
        tags = []

        for i, word in enumerate(sent):
            # Определим предыдущий тег
            if i == 0:
                prev_t = 'START'
            else:
                prev_t = tags[i - 1]

            # Исправленный алгоритм, max_prob и best_tag обновляем для каждого слова
            max_prob = 0
            best_tag = 'UNK'
            for t in self.tm.tags():
                prob = self.em(word, t) * self.tm(t, prev_t)
                if prob > max_prob:
                    max_prob, best_tag = prob, t
            tags.append(best_tag)

        return list(zip(sent, tags))

# Обучение теггера и сравнение точности для разных корпусов

### Загрузим корпуса, на которых будем обучать теггер

In [6]:
# conll2000 корпус
conll2000.ensure_loaded()

train_sents_conll2000 = conll2000.tagged_sents()[:8000]

# UD_English-EWT корпус
train_sents_ewt_ud = []
with open('./UD_English-EWT/en_ewt-ud-train.conllu') as train_set:
    for tokens in parse(train_set.read()):
        train_sents_ewt_ud.append([(token['form'], token['upostag']) for token in tokens])

### Обучим теггеры

In [7]:
# Теггер, обученный на conll2000
em_conll2000 = EmissionModel(train_sents_conll2000)
tm_conll2000 = TransitionModel([[tag for word, tag in sent] for sent in train_sents_conll2000])
tagger_conll2000 = BigramPOSTagger(em_conll2000, tm_conll2000)

In [8]:
# Теггер, обученный на UD_English-EWT
em_ewt_ud  = EmissionModel(train_sents_ewt_ud)
tm_ewt_ud = TransitionModel([[tag for word, tag in sent] for sent in train_sents_ewt_ud])
tagger_ewt_ud = BigramPOSTagger(em_ewt_ud, tm_ewt_ud)

### Сравним точность теггеров

In [9]:
test_sents_conll2000 = conll2000.tagged_sents()[8000:]
print(f'Точность теггера, обученного на conll2000: {accuracy(test_sents_conll2000, tagger_conll2000)}')

test_sents_ewt_ud = []
with open('./UD_English-EWT/en_ewt-ud-test.conllu') as test_set:
    for tokens in parse(test_set.read()):
        test_sents_ewt_ud.append([(token['form'], token['upostag']) for token in tokens])
print(f'Точность теггера, обученного на UD_English-EWT: {accuracy(test_sents_ewt_ud, tagger_ewt_ud)}')

Точность теггера, обученного на conll2000: 0.8722227025157776
Точность теггера, обученного на UD_English-EWT: 0.8385065944136749


Точность теггера, обученного на корпусе conll2000 оказалась выше, чем на UD_English-EWT.